# Set Required Args to Activate Modules

In [ ]:
from workflow.scripts.resources import Resources

# single resource

In [ ]:
# Iterate over provinces for both solar and wind resources
resource_types = ['wind', 'solar']
provinces=['BC','AB','SK','ON','NS']
for province_code in provinces:
    for resource_type in resource_types:
        required_args = {
            "config_file_path": 'config/config.yml',
            "province_short_code": province_code,
            "resource_type": resource_type
        }
        
        # Create an instance of Resources and execute the module
        resource_module = Resources(**required_args)
        resource_module.execute_module()
        